In [1]:
import pandas as pd
bad = pd.read_csv('../data/bad/bad.csv')
normal=pd.read_csv('../data/neautral/neutral.csv')
hiniku=pd.read_csv('../data/hiniku/hiniku.csv')

In [2]:
badwordlist = list(bad.word)#悪口単語をリストで抽出

In [49]:
'''
[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [文章中に現れる悪口数]
'''

# df = bad.copy()
df = pd.concat([hiniku, normal])
df = pd.concat([bad, df])
import sys
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
import numpy as np
from janome.tokenizer import Tokenizer
import janome

tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_bad_word_count(text, badwordlist):

    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in badwordlist:
            count+=1
    for w in badwordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_bad_feature(df, badwordlist):
    col=["content","mophologics_num", "bad_word_num", "simple_bad_flg","target"]
    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in df.iterrows():
        text = r[1].content
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        count,flg = get_bad_word_count(text, badwordlist)
        
        newdf = pd.DataFrame(np.array([text, num, count,flg,target]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
    return basedf
df =make_bad_feature(df, badwordlist)

In [50]:
df

,content,mophologics_num,bad_word_num,simple_bad_flg,target
0,ガイジ,1,1,1,1
0,くどい,1,1,1,1
0,偽物 ただの釣り動画,6,2,1,1
0,振り付けダサイ,2,0,1,1
0,男きも,3,0,1,1
...,...,...,...,...,...
0,絶対結婚するぅぅ,4,0,0,0
0,ええこや,4,0,0,0
0,教養のあるゴリラやな,6,1,1,0
0,なんか漏れたかなー これは惜しい,9,0,0,0


In [51]:
X = df[["mophologics_num", "bad_word_num", "simple_bad_flg"]]
y = df.target

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
svm = SVC(C=1.0, random_state=42)
svm.fit(x_train, y_train)

pred_train = svm.predict(x_train)
pred_test = svm.predict(x_test)
print(f1_score(y_train, pred_train))
print(f1_score(y_test, pred_test))

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
svm = SVC(C=1.0, random_state=42)
model = OneVsRestClassifier(svm)
model.fit(x_train, y_train)

pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
# print(f1_score(y_train, pred_train))
# print(f1_score(y_test, pred_test))

In [57]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

'''
mxm,ここでは3x3のマトリックスを作成する
そこからf1値を計算する（同ディレクトリの画像参照）

'''

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [42]:
def print_miss_pred(true_y, pred):
    arr = []
    pred_judge=[]
    true_judge=[]
    for i in range(len(pred)):
        if pred[i]!=true_y.values[i]:
            arr.append(i)
            pred_judge.append(pred[i])
            true_judge.append(true_y.values[i])

    def print_str_label(num):
        if num==0:
            return "ニュートラル"
        if num==1:
            return "悪口"
        if num==2:
            return "皮肉"
        return "バグあり"

    for i,idx in enumerate(arr):
        display(df.iloc[idx])
        print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
        print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
        print("------------")

In [44]:
print_miss_pred(y_test, pred_test)

content            振り付けダサイ
mophologics_num          2
bad_word_num             0
simple_bad_flg           1
target                   1
Name: 0, dtype: object

true label  * 1 悪口
pred label  * 2 皮肉
------------


content            人間のクズが
mophologics_num         4
bad_word_num            1
simple_bad_flg          1
target                  1
Name: 0, dtype: object

true label  * 1 悪口
pred label  * 2 皮肉
------------


content            ハゲやん
mophologics_num       2
bad_word_num          1
simple_bad_flg        1
target                1
Name: 0, dtype: object

...

true label  * 2 皮肉
pred label  * 1 悪口
------------


content            あーころす
mophologics_num        2
bad_word_num           1
simple_bad_flg         1
target                 1
Name: 0, dtype: object

true label  * 2 皮肉
pred label  * 1 悪口
------------


content            なら自分で描いてネットに載せろ9ず
mophologics_num                   10
bad_word_num                       0
simple_bad_flg                     1
target                             1
Name: 0, dtype: object

true label  * 1 悪口
pred label  * 2 皮肉
------------


2